# 🔖Google COLAB Mounting
* 아래 코드는 Google COLAB에서 Drive에 접근하기 위해 필요한 코드로, local에서 코드를 돌릴 때는 필요하지 않은 코드입니다. (애초에 돌아가지 않습니다.)
* 코드를 돌린 뒤 사용허가 창이 뜨면 전부 '확인'하고 넘어가시면 됩니다.

In [96]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 🔖1. Import Dataset & Check Variables

## 1.1 Directory Setting

In [97]:
path_load = "/content/drive/My Drive/12/"
# 데이터 파일이 저장된 경로를 입력해 주세요. (본인이 데이터를 저장한 경로로 입력해야 합니다.)
# Google Drive 내에 있는 폴더라면 "/content/drive/My Drive/" 뒤에 폴더 경로를 적어야 합니다.

In [98]:
import os
os.chdir(path_load)

## 1.2 Import CSV File
* 한글 파일을 불러올 때 글자가 깨져 보이거나 encoding 관련 에러 메시지가 뜰 경우 encoding="cp949" 설정을 "utf-8" 혹은 "euc-kr"로 바꿔 보세요.
* 자세한 내용은 올려드린 "Manual - 한글 텍스트 파일 데이터 깨질 경우 해결책.docx"을 참고하시면 됩니다.

In [99]:
import pandas as pd
data = pd.read_csv("samsung.csv", encoding="utf-8")

In [100]:
data.head(3)

,id,rate_date,언론사,기고자,rate_comment,통합 분류1,통합 분류2,통합 분류3,사건/사고 분류1,사건/사고 분류2,사건/사고 분류3,인물,위치,기관,키워드,특성추출(가중치순 상위 50개),본문,URL,분석제외 여부
0,7.101201e+06,20240602,디지털타임스,김나인,"삼성전자, `AI폰` 시장 장악에 폴더블폰 주춤? `갤Z6`로 반전 꾀할까",IT_과학>모바일,경제>유통,IT_과학>콘텐츠,NaN,NaN,NaN,"블룸버그인텔리전스,블룸버그","중국,장악,인터내셔널데이터코퍼레이션","샤오미,애플,중국,BI,샤오미14,삼성전자,모토로라,화웨이,삼성","삼성전자,AI폰,시장,장악,폴더블폰,반전,삼성전자,출시,S24,시리즈,생성,인공,지...","스마트폰,폴더블폰,생성형,삼성전자,점유율,폴더블,ai,중국,화웨이,카운터포인트리서치...",삼성전자가 올 초 출시한 '갤럭시S24' 시리즈로 생성형 인공지능(AI) 지원 스마...,http://www.dt.co.kr/contents.html?article_no=2...,NaN
1,1.010010e+07,20240602,스포츠서울,표권향 기자,"“오늘은 AI 요리사” 삼성전자, 인덕션 식기세척기 구매 혜택 ‘인 식바꿈’ 프로모...",IT_과학>모바일,경제>서비스_쇼핑,경제>반도체,NaN,NaN,NaN,NaN,화력,"유일,삼성닷컴,삼성전자,삼성","AI,요리사,삼성전자,인덕션,식기세척기,식바꿈,구매,혜택,프로모션,진행,삼성전자,이...","인덕션,ai,비스포크,삼성전자,세척기,식기세척기,소비자,식바꿈,500만,시니어,인피...",삼성전자가 이달 1일부터 9월30일까지 AI 인덕션 및 식기세척기를 풍성한 혜택과 ...,https://www.sportsseoul.com/news/read/1434093,NaN
2,4.100958e+06,20240602,EBN,EBN 이남석 기자 (leens0319@ebn.co.kr),"“AI로 편하게 요리하자” 삼성전자, ‘인 식바꿈’ 프로모션 실시",IT_과학>모바일,경제>서비스_쇼핑,경제>산업_기업,NaN,NaN,NaN,NaN,화력,"유일,세척기삼성전자,삼성닷컴,삼성전자,삼성","AI,요리,삼성전자,식바꿈,프로모션,인덕션,식기,세척기,교체,요리,시작,걱정,AI,...","인덕션,비스포크,세척기,삼성전자,ai,소비자,식바꿈,식기세척기,신혼여행,오염도,삼성...","인덕션, 식기세척기 교체로 요리 시작부터 끝까지 편하게 \n넘침 걱정 없는 AI 끓...",NaN,NaN


# 🔖2. Sentiment Dictionary
* 출처: KNU 감성어분석사전 (https://github.com/park1200656/KnuSentiLex)
* SentiWord_Dict.txt 파일을 저장해 text 파일로 불러오면 됩니다.
* 해당 사전은 단어의 감성을 -2(부정적)에서 2(긍정적)까지 5점 척도로 수치화합니다.

## 2.1 Import Sentiment Dictionary

In [101]:
# Import Dictionary txt file into dataframe
dictionary = pd.read_csv("SentiWord_Dict.txt", sep='\t', lineterminator='\n', header=None)
dictionary.columns = ['word', 'polarity']

## 2.2 Setting Function for Matching Score

* 우선, Python의 dictionary 타입은
```
dict_sample = {index1:value1, index2:value2}
```
와 같은 형태로 이루어져,
```
dict_sample[index1]
```
과 같이 indexing을 하면 값으로 value1을 돌려줍니다.

* 이를 이용해 위의 sentiment dictionary를 dictionary 타입으로 만들어 단어별 점수 접근을 용이하게 합니다.

In [102]:
dict_dictionary = dictionary.set_index('word')['polarity'].to_dict()

# sample: sentiment score of '멋있다'
dict_dictionary['좋다']

2.0

* 이제 이 dictionary를 이용해 각 단어의 score를 return하는 함수를 만들어 줍니다.
* 어떠한 단어 word가 dict_dictionary에 있을 경우 그 점수를, 없을 경우 0을 반환하는 함수 sentiment_score는 다음과 같습니다.

In [103]:
# Define function for matching word in dictionaries
def sentiment_score(word):
  if word in dict_dictionary.keys():
    return(dict_dictionary[word])
  else:
    return(0)

* 만들어진 sentiment_score 함수를 이용해 여러 단어의 감성 점수를 합산해 주는 sentiment_scores를 만들면 다음과 같습니다.
* word_list 안에 포함된 모든 word의 점수값을 합산해 반환합니다.

In [104]:
# Define function for sentiment score sum
def sentiment_scores(word_list):
  return(sum([sentiment_score(word) for word in word_list]))

In [107]:
# 이때 '재밌다'의 index에 점수는 2.0이라는 value를 입력해 줄 수 있습니다.
dict_dictionary['성장'] = 2.0
dict_dictionary['1위'] = 2.0
dict_dictionary['선도'] = 2.0
dict_dictionary['호평'] = 2.0

In [108]:
# example
print(sentiment_score('좋다')) # 2.0
print(sentiment_score('성장')) # 2.0
print(sentiment_scores(['좋다', '성장'])) # 2.0 + 2.0 = 4.0

2.0
2.0
4.0


# 🔖3. Data Cleaning for Text Analysis
* Python text cleaning 참고자료: https://wikidocs.net/21694

In [109]:
# 한국어 자연어처리를 위한 패키지
# Colab이 아닌 local에서 설치할 때는 설치 과정이 조금 더 복잡하니 가능한 colab에서 수행하시기 바랍니다.
!pip install konlpy
import konlpy

## 3.1 특수문자 제거

* 정규표현식(regex)를 이용해 의미를 갖지 않는 특수문자(예: 온점, 반점, 따옴표 등)를 제외하고 한글, 영어, 숫자 등만 남깁니다.
* 파이썬 정규표현식 참고자료: https://wikidocs.net/1642



English version:
```
import re
df = df.apply(lambda x: re.sub(r'[^\w\d\s]', '', x))
df.head()
```

In [110]:
import re
import numpy as np

In [111]:
re.sub(r'[^\w\d\s\ㄱ-힣]', ' ', "** 예시 텍스트** (문장에 필요하지 않은 성분은 공백으로 대체합니다.)")

'   예시 텍스트    문장에 필요하지 않은 성분은 공백으로 대체합니다  '

In [112]:
data['rate_comment'][:10]

0            삼성전자, `AI폰` 시장 장악에 폴더블폰 주춤? `갤Z6`로 반전 꾀할까
1    “오늘은 AI 요리사” 삼성전자, 인덕션 식기세척기 구매 혜택 ‘인 식바꿈’ 프로모...
2                 “AI로 편하게 요리하자” 삼성전자, ‘인 식바꿈’ 프로모션 실시
3             'AI폰' 끌고 키워 온 삼성전자... 폴더블 시장에선 추격 매서운 중국
4                     삼성전자 ‘AI 원년’ 6개월 한종희 “열심히 하고 있다”
5                   삼성전자, 글로벌 1Q AI폰 점유율 58.4%로 압도적 1위
6                 "5만원 더"...민팃, 삼성전자와 '갤럭시 AI 추가보상' 시행
7                     삼성전자 글로벌 AI폰 시장 장악...1분기 점유율 58%
8    “삼성전자 갤럭시 S24 시리즈, AI 스마트폰 시장 지배” 1분기 생성형 AI폰 ...
9                           민팃, 삼성전자와 ‘갤럭시 AI 추가보상’ 시행
Name: rate_comment, dtype: object

In [113]:
data['rate_comment_rm'] = data['rate_comment'].apply(lambda x: re.sub(r'[^\w\d\s\ㄱ-힣]', ' ', str(x)) if x!= np.nan else x)
# "lambda x: function(x)"는 x에 대한 간단한 함수를 수행합니다.
# 위의 함수는 x가 비어 있는 missing 상태가 아니라면 (if x!=np.nan) re.sub(r'[^\w\d\s\ㄱ-힣]', ' ', str(x))을 이용해 텍스트의 특수문자를 제거하고, 아니면 (else) x를 그대로 두라는 의미입니다.
# 여기서 x는 data['rate_comment']의 각 줄의 값을 의미합니다.

data['rate_comment_rm'][:10]

0            삼성전자   AI폰  시장 장악에 폴더블폰 주춤   갤Z6 로 반전 꾀할까
1     오늘은 AI 요리사  삼성전자  인덕션 식기세척기 구매 혜택  인 식바꿈  프로모...
2                  AI로 편하게 요리하자  삼성전자   인 식바꿈  프로모션 실시
3              AI폰  끌고 키워 온 삼성전자    폴더블 시장에선 추격 매서운 중국
4                     삼성전자  AI 원년  6개월 한종희  열심히 하고 있다 
5                   삼성전자  글로벌 1Q AI폰 점유율 58 4 로 압도적 1위
6                  5만원 더    민팃  삼성전자와  갤럭시 AI 추가보상  시행
7                     삼성전자 글로벌 AI폰 시장 장악   1분기 점유율 58 
8     삼성전자 갤럭시 S24 시리즈  AI 스마트폰 시장 지배  1분기 생성형 AI폰 ...
9                           민팃  삼성전자와  갤럭시 AI 추가보상  시행
Name: rate_comment_rm, dtype: object

## 3.2 토큰화 (Tokenization) & 표제어 추출 (Lemmatization)
* 토큰화(Tokenization): 문장 내에서 공백으로만 나뉘어 있는 단어(어절) 단위를 자르고,
* 표제어 추출(Lemmatization): 각 단어의 원형을 찾아 줍니다. (e.g., "자르고"의 원형은 "자르다")


* 영어와 한국어 사이에 패키지 차이가 있음 (한글은 konlpy, 영어는 nltk 사용)

English version - Lemmatization

```
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.tokenize import WordPunctTokenizer
df_token = df.apply(lambda x: word_tokenize(x))

nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
WL = WordNetLemmatizer()
tokens = tokens.apply(lambda x: [WL.lemmatize(word, 'n') for word in x])
```

English Version - Cleaning Stopwords
```
nltk.download("stopwords")
from nltk.corpus import stopwords
sw_list = stopwords.words("english")
tokens = tokens.apply(lambda x: [word for word in x if word not in sw_list])
```

In [114]:
from konlpy.tag import Okt
okt = Okt()

In [115]:
sample_word = data['rate_comment_rm'].tolist()[0]

print(sample_word)
print(okt.morphs(sample_word))
print(okt.morphs(sample_word, norm=True)) # 맞춤법, 띄어쓰기 보정
print(okt.morphs(sample_word, stem=True)) # 단어의 원형 탐색
print(okt.morphs(sample_word, norm=True, stem=True))

삼성전자   AI폰  시장 장악에 폴더블폰 주춤   갤Z6 로 반전 꾀할까
['삼성', '전자', 'AI', '폰', '시장', '장악', '에', '폴', '더블', '폰', '주춤', '갤', 'Z', '6', '로', '반전', '꾀', '할까']
['삼성', '전자', 'AI', '폰', '시장', '장악', '에', '폴', '더블', '폰', '주춤', '갤', 'Z', '6', '로', '반전', '꾀', '할까']
['삼성', '전자', 'AI', '폰', '시장', '장악', '에', '폴', '더블', '폰', '주춤', '개다', 'Z', '6', '로', '반전', '꾀', '하다']
['삼성', '전자', 'AI', '폰', '시장', '장악', '에', '폴', '더블', '폰', '주춤', '개다', 'Z', '6', '로', '반전', '꾀', '하다']


In [116]:
# Apply the lemmatization function (앞과 같은 방법으로 함수 적용)
data['rate_comment_rm_lm'] = data['rate_comment_rm'].apply(lambda x: okt.morphs(x, norm=True, stem=True))

# 🔖4. Sentiment Analysis: matching Sentiment Dictionary scores
* 앞에서 만들어 둔 sentiment_scores() 함수를 이용해 점수를 계산해 줍니다.

In [117]:
# Apply sentiment scores (sum) function
data['rate_sentiscore'] = data['rate_comment_rm_lm'].apply(lambda x: sentiment_scores(x))
data[['rate_comment', 'rate_sentiscore']].head(3)

,rate_comment,rate_sentiscore
0,"삼성전자, `AI폰` 시장 장악에 폴더블폰 주춤? `갤Z6`로 반전 꾀할까",-1.0
1,"“오늘은 AI 요리사” 삼성전자, 인덕션 식기세척기 구매 혜택 ‘인 식바꿈’ 프로모...",1.0
2,"“AI로 편하게 요리하자” 삼성전자, ‘인 식바꿈’ 프로모션 실시",2.0


In [122]:
print(sentiment_score('성장하다'))
print(sentiment_score('강세'))
print(sentiment_score('좋다'))
print(sentiment_score('주춤'))

print(sentiment_scores(okt.morphs(sample_word, norm=True, stem=True)))

2.0
1.0
2.0
-1.0
-1.0


In [125]:
data[['rate_comment','rate_sentiscore']]

,rate_comment,rate_sentiscore
0,"삼성전자, `AI폰` 시장 장악에 폴더블폰 주춤? `갤Z6`로 반전 꾀할까",-1.0
1,"“오늘은 AI 요리사” 삼성전자, 인덕션 식기세척기 구매 혜택 ‘인 식바꿈’ 프로모...",1.0
2,"“AI로 편하게 요리하자” 삼성전자, ‘인 식바꿈’ 프로모션 실시",2.0
3,'AI폰' 끌고 키워 온 삼성전자... 폴더블 시장에선 추격 매서운 중국,0.0
4,삼성전자 ‘AI 원년’ 6개월 한종희 “열심히 하고 있다”,2.0
...,...,...
954,"냉장고에 '보안 최고등급'? 삼성전자, AI로 세계 최초 인증",2.0
955,노트북도 AI가 대세...삼성전자 갤럭시북 vs LG 그램,0.0
956,"삼성전자, AI 노트북 ‘갤럭시 북4 시리즈’ 국내 판매 10만대 돌파",0.0
957,[특징주] 美 AI 반도체 훈풍에...삼성전자 SK하이닉스 강세,1.0


In [124]:
data['rate_sentiscore'].mean()

0.32638164754953075

### Additional Notes. Add Personal Words to Sentiment Dictionary

In [ ]:
# "재미있다"의 점수는 2.0으로 있는데, "재밌다"의 점수는 입력되지 않아서 0으로 나오는 상황
sentiment_score('재미있다'), sentiment_score('재밌다')

(2.0, 0)

In [ ]:
# dictionary에 '재밌다'를 찾아보면 없는 단어기 때문에 Error message가 뜸
dict_dictionary['재밌다']

KeyError: '재밌다'

In [ ]:
# 다시 계산해 보면 입력한 대로 2.0이 나옵니다.
sentiment_score('재미있다'), sentiment_score('재밌다')

(2.0, 2.0)

In [ ]:
data['rate_sentiscore'] = data['rate_comment_rm_lm'].apply(lambda x: sentiment_scores(x))
data[['rate_comment', 'rate_sentiscore']].head(3)

,rate_comment,rate_sentiscore
0,너무 재밌어서 소름돋음,0.0
1,이도현 그래서 제대가 언제라고?,0.0
2,최민식은 역시 최민식이도현은 지금껏 가장 멋있었고김고은의 굿 연기는 현대적 감각이 ...,2.0
